In [1]:
import imaplib
import email

from bs4 import BeautifulSoup
import gc
import os
import requests
import httpx
import re
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ChromeOptions, Chrome
import random
import json
import logging
import pandas as pd
from string import ascii_letters, punctuation
from fake_useragent import UserAgent

logger = logging.getLogger("selenium")
logger.setLevel(logging.INFO)

android_ua = "Mozilla/5.0 (Linux; Android 14; LM-X420) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.210 Mobile Safari/537.36"


def load_cookies():
    with open("cookies.json") as f:
        cooks = json.load(f)
    return cooks


def get_options(mobile=False, headless=False):
    options = ChromeOptions()
    if mobile:
        mobile_emulation = {
            "deviceMetrics": {"width": 720, "height": 1280, "pixelRatio": 3.0},
            # "Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexus 5 Build/JOP40D) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166 Mobile Safari/535.19",
            "userAgent": android_ua,
            "clientHints": {"platform": "Android", "mobile": True},
        }
        options.add_experimental_option("mobileEmulation", mobile_emulation)
    if headless:
        options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    # this argument disables the welcome screen
    options.add_argument("--disable-fre")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument("--show")
    # options.add_argument("--window-size=1080,1920")
    # options.add_experimental_option("excludeSwitches", ["enable-automation"])
    # options.add_experimental_option('useAutomationExtension','False')
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument(f"--proxy-server={get_proxies()[0]}")
    # options.add_argument(f'--proxy-server=http://72.10.160.90:3995')
    # options.add_experimental_option(
    #     "prefs",
    #     {
    #         "profile.default_content_setting_values.media_stream_mic": 1,
    #         "profile.default_content_setting_values.media_stream_camera": 1,
    #         "profile.default_content_setting_values.geolocation": 0,
    #         "profile.default_content_setting_values.notifications": 1,
    #     },
    # )
    # options.add_argument(
    #     "--user-data-dir=C:/Users/Dunes/AppData/Local/Google/Chrome/User Data"
    # )
    return options


def nav_update(driver):
    try:
        pass
    except Exception as e:
        pass
    # driver.maximize_window()
    x = driver.execute_script("return window.pageYOffset")
    new = 1
    count = 0
    while x != new:
        gc.collect()
        try:
            driver.execute_script("return window.scrollBy(0,1000)")
            time.sleep(2)
            new = x
            x = driver.execute_script("return window.pageYOffset")
        except:
            pass
        x = driver.execute_script("return window.pageYOffset")
        if x == new or count > 25:
            # count+=1
            # print(x,new)
            # print(count)
            # if count>3:
            break
        count += 1


def get_proxies():
    proxy_url = "https://api.proxyscrape.com/v2/?request=displayproxies&protocol=http&timeout=10000&country=all&ssl=yes&anonymity=elite"
    r = requests.get(proxy_url)
    proxies = [i.strip() for i in r.text.splitlines()]
    with open("proxies.txt", "w") as f:
        for proxy in proxies:
            f.write(f"{proxy}\n")
    random.shuffle(proxies)
    print(proxies[0])
    return proxies


def getOTP(userEmail: str, password: str) -> str:
    # Outlook IMAP settings
    outlook_server = "outlook.office365.com"

    # Connect to Outlook's IMAP server
    mail = imaplib.IMAP4_SSL(outlook_server)
    mail.login(userEmail, password)

    # Select the mailbox you want to access
    mail.select("inbox")

    # Search for all emails in the inbox
    status, messages = mail.search(None, "ALL")
    messages = messages[0].split()

    # Get the latest email
    latest_email_id = messages[-1]

    # Fetch the email by ID
    status, msg_data = mail.fetch(latest_email_id, "(RFC822)")
    raw_email = msg_data[0][1]

    # Parse the raw email
    msg = email.message_from_bytes(raw_email)

    # Extract the HTML part of the email
    for part in msg.walk():
        content_type = part.get_content_type()

        # Check for text/html parts
        if "text/html" in content_type:
            # Get the HTML content
            html_content = part.get_payload(decode=True).decode("utf-8")

    # Parse the HTML part with BeautifulSoup
    soup = BeautifulSoup(html_content, "html5lib")

    # Find the confirmation code element by its style attribute
    confirmation_code_element = soup.find(
        "td",
        style="padding:10px;color:#565a5c;font-size:32px;font-weight:500;text-align:center;padding-bottom:25px;",
    )

    # Extract the confirmation code
    if confirmation_code_element:
        confirmation_code = confirmation_code_element.get_text(strip=True)
        code = confirmation_code
    else:
        code = "Confirmation code element not found."

    # Close the connection
    mail.logout()

    # return the code
    return code, soup

In [2]:
driver = uc.Chrome(options=get_options(mobile=False, headless=False))

72.10.164.178:1689


In [11]:
for cookies in load_cookies():
    cookies["domain"] = ".tiktok.com"
    print(cookies)
    driver.add_cookie(cookies)

{'domain': '.tiktok.com', 'expiry': 1713492044, 'httpOnly': False, 'name': 'msToken', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'zXngqhMOBAJjkt3_0LxnI1OAYCnW2L1GeQeydiEJCWbGPmzicmJ-ncoMg1jxfwTuiFt2kJBkkttjG6Ro7rLTtk2ktSex59vlrvXYAoAi_FuyExL1-Nfe3rK0gcm-IxpeP_9HUwHIJ6i8psE='}
{'domain': '.tiktok.com', 'expiry': 1737251989, 'httpOnly': True, 'name': 'odin_tt', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'd3315003aebeddb4bb41f9bd5c5a6a9f6e9e1b42c8ad60d380dd343fe8002613dba384dad6b613f48d705a5b6661538f4d1c0a6e7f3110624a4378c369118886915449eac421f604b3721ff33c528fc1'}
{'domain': '.tiktok.com', 'expiry': 1706147990, 'httpOnly': False, 'name': 'perf_feed_cache', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '{%22expireTimestamp%22:1705885200000%2C%22itemIds%22:[%227315404328829881605%22%2C%227300214016146722080%22%2C%227320295069636988203%22]}'}
{'domain': '.tiktok.com', 'httpOnly': False, 'name': 'passport_fe_beating_status', 'path': '/', 'sameSite': '

In [ ]:
proxies = get_proxies()
proxy = "http://" + proxies[0]
url = "https://myexternalip.com/raw"
r = httpx.get(url, proxies={"http://": proxy, "https://": proxy})
r

In [ ]:
driver.quit()
del driver

In [ ]:
driver.get("https://httpbin.org/ip")
driver.find_element(By.TAG_NAME, "body").text

In [ ]:
with open("cookis.json", "w") as f:
    json.dump(driver.get_cookies(), f)

In [ ]:
user_info = pd.read_excel("testing list.xlsx")
user_info = json.loads(user_info.to_json(orient="index"))
user_info

In [ ]:
def get_info(num):
    return user_info[str(num)]


email, password, recovery = get_info().values()
email, password

In [ ]:
# create tiktok signup url assigned to url
url = "https://www.tiktok.com/signup"
driver.get(url)
url = "https://www.tiktok.com/signup/phone-or-email/email"

driver.get(url)

In [4]:
# driver.get("https://www.tiktok.com/login")
driver.get("https://www.tiktok.com/login/phone-or-email/email")

In [ ]:
def get_value(typ="m"):
    if typ == "m":
        val = random.randint(0, 11)
        # val = f"Month-options-item-{val}"
    elif typ == "y":
        val = random.randint(18, 40)
        # val = f"Year-options-item-{val}"
    elif typ == "d":
        val = random.randint(0, 28)
        # val = f"Day-options-item-{val}"
    else:
        val = 2
    return val


val = get_value("y")
val

In [ ]:
def get_age():
    age = driver.find_elements(By.CLASS_NAME, "tiktok-1pp5msd-DivSelector")
    values = ["m", "d", "y"]
    for idx, i in enumerate(age):
        i.click()
        vals = i.find_elements(By.CLASS_NAME, "tiktok-x376y3-DivOption")
        val = get_value(values[idx])
        print(val)
        vals[val].click()

In [ ]:
def create_account():
    account_email = "".join(random.choices(ascii_letters, k=10)) + "@gnail.com"
    password = "".join(random.choices(
        ascii_letters + punctuation + "1234567890", k=10))
    if not os.path.exists("accounts.txt"):
        with open("accounts.txt", "w") as f:
            f.write("Email, Password\n")
            f.write(f"{account_email} , {password}\n")
    else:
        with open("accounts.txt", "a") as f:
            f.write(f"{account_email} , {password}\n")
    return account_email, password


base_url = "https://api.mail.tm"
email, password = create_account()
email, password

In [ ]:
from collections import OrderedDict
from string import ascii_letters, punctuation


class MailTM:
    def __init__(self):
        self.email = None
        self.password = None
        self.base_url = "https://api.mail.tm"
        self.headers = {
            "Accept": "application/json",
        }
        self.json_path = "accounts.json"
        self.domains = None
        self.account_details = {}
        self.has_accounts = self.has_creds()

    def __repr__(self):
        return f"MailTM({self.email} , {self.password})"

    @property
    def get_domains(self):
        r = requests.get(f"{self.base_url}/domains")
        self.domains = r.json().get("hydra:member", {})
        return self.domains

    def has_creds(self):
        if not os.path.exists(self.json_path):
            return False
        with open(self.json_path) as file:
            creds = json.load(file)
        if len(creds) > 0:
            self.account_details = creds
            return True
        return False

    @property
    def random_info(self):
        account_email = (
            "".join(random.choices(ascii_letters, k=10))
            + "@"
            + self.get_domains[0].get("domain", "")
        )
        if not "@" in account_email:
            raise ValueError("domain not found")
        password = "".join(
            random.choices(ascii_letters + ",.?!@#$%^&*" + "1234567890", k=10)
        )
        if not os.path.exists("accounts.txt"):
            with open("accounts.txt", "w") as f:
                f.write("Email, Password\n")
                f.write(f"{account_email} , {password}\n")
        else:
            with open("accounts.txt", "a") as f:
                f.write(f"{account_email} , {password}\n")
        self.email = account_email
        self.password = password
        return account_email, password

    def create_account(self, num=2, save=True, random_account=False):
        if random_account:
            self.account_details = {}
        for i in range(num):
            self.random_info
            r = requests.post(
                f"{self.base_url}/accounts",
                json={"address": self.email, "password": self.password},
            )
            account_details = r.json()
            account_details["password"] = self.password
            self.account_details[account_details["id"]] = account_details
        if save:
            if self.has_accounts:
                with open(self.json_path, "r") as f:
                    self.account_details.update(json.load(f))
            with open(self.json_path, "w") as f:
                json.dump(self.account_details, f, indent=4)
        return self.accounts()

    def accounts(self, save=True):
        self.get_tokens()
        if self.has_accounts:
            if save:
                with open(self.json_path, "w") as f:
                    json.dump(self.account_details, f, indent=4)
            return self.account_details
        self.create_account()
        if os.path.exists(self.json_path):
            with open(self.json_path, "r") as f:
                self.account_details.append(json.load(f))
        return self.account_details

    def get_tokens(self):
        for idx, account_info in self.account_details.items():
            if account_info.get("token"):
                continue
            email = account_info["address"]
            password = account_info["password"]
            r = requests.post(
                f"{self.base_url}/token", json={"address": email, "password": password}
            )
            account_info.update(r.json())
            self.account_details[idx] = account_info
        print("Tokens Generated")
        return self.account_details

In [ ]:
tm = MailTM()
tm

In [ ]:
tm.create_account(num=1, save=False, random_account=True)
# tm.accounts()

In [ ]:
accounts = tm.accounts(save=True)
for idx, account in accounts.items():
    email = account["address"]
    password = account["password"]
    token = account["token"]
    break
email, password, token
# accounts

In [ ]:
r = requests.get(
    f"{tm.base_url}/messages",
    headers={"Authorization": f"Bearer {token}"},
    json={"address": email, "password": password},
)
# Send post request with email and password in request body


# r = requests.get(f'{base_url}/domains', json = {'address':email,'password':password})
# print(r.json())
def get_otp(email, password, token):
    r = requests.get(
        f"{tm.base_url}/messages",
        headers={"Authorization": f"Bearer {token}"},
        json={"address": email, "password": password},
    )
    messages = r.json().get("hydra:member", [])[0]["intro"]
    date = r.json().get("hydra:member", [])[0]["createdAt"]
    return messages, date, re.search("\d{6}", messages).group(0)


# get_otp()

In [ ]:
driver = uc.Chrome(options=get_options(mobile=False, headless=False))
driver.delete_all_cookies()


driver.get("https://www.tiktok.com")
time.sleep(3)
driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)

driver.get("https://www.tiktok.com/signup")
driver.refresh()

driver.get("https://www.tiktok.com/signup/phone-or-email/email")

In [ ]:
em = "Enrikkodra461253@outlook.com"
pwd = "LAj461ciJS."
opwd = "LAj461ciJS"
uname = "revolvingmanana"

In [ ]:
email_elem = driver.find_element(By.TAG_NAME, "input")
email_elem.clear()
for i in em:
    email_elem.send_keys(i)

In [ ]:
driver.find_element(By.ID, "email-consent").click()

In [ ]:
driver.find_element(By.XPATH, '//button[@type="submit"]').click()

In [ ]:
def input_details():
    inputs = driver.find_elements(By.TAG_NAME, "input")
    keys = [em, pwd]
    for idx, elem in enumerate(inputs[:2]):
        print(elem.get_attribute("placeholder"))
        elem.clear()
        elem.send_keys(keys[idx])
        time.sleep(2)
    return inputs[-2]
    # code = input("Enter code...")
    # if code:
    #     inputs[-2].send_keys(code)


get_age()
otp = input_details()
otp

In [ ]:
import re

tp = getOTP(em, opwd)
key = re.findall("\d{6}", str(tp[1]))

In [14]:
driver.find_element(By.LINK_TEXT, "Upload").click()

In [ ]:
send_code = driver.find_element(
    By.XPATH, '//button[@data-e2e="send-code-button"]'
).click()
send_code = driver.find_element(
    By.XPATH, '//button[@data-e2e="send-code-button"]'
).click()

In [ ]:
# otp_key = get_otp(email, password, token)[-1]
otp_key = "539644"


otp.clear()


otp.send_keys(otp_key)


otp_key

In [ ]:
# Post signup
if "create-username" in driver.current_url:
    print("i")

In [ ]:
username = driver.find_element(By.TAG_NAME, "input")
# username.get_attribute("placeholder")
username.clear()
username.send_keys("Revolvingmanana")

In [ ]:
driver.refresh()

In [ ]:
submit = driver.find_element(By.XPATH, '//button[@type="submit"]')
submit.click()

In [ ]:
login_url = "https://www.tiktok.com/login/phone-or-email/email"
driver.get(login_url)

In [ ]:
login_elems = driver.find_elements(By.TAG_NAME, "input")
keys = [email, password]

for idx, elem in enumerate(login_elems):
    elem.clear()
    elem.send_keys(keys[idx])
driver.find_element(By.XPATH, '//button[@type="submit"]').click()

In [ ]:
import json